In [2]:
def initialize_matrix(n):
    """Initialize an n x n matrix with zeros."""
    return [[0] * n for _ in range(n)]

def is_pair(base1, base2):
    """Check if two bases form a valid pair."""
    return (base1 == 'A' and base2 == 'U') or (base1 == 'U' and base2 == 'A') or \
           (base1 == 'G' and base2 == 'C') or (base1 == 'C' and base2 == 'G')

def nussinov(seq):
    """Nussinov algorithm to predict RNA secondary structure."""
    n = len(seq)
    dp = initialize_matrix(n)

    # Fill the DP table
    for k in range(1, n): # gap size
        for i in range(n - k):
            j = i + k
            # Recurrence relation
            dp[i][j] = max(
                dp[i + 1][j], # Unpaired base at i
                dp[i][j - 1], # Unpaired base at j
                dp[i + 1][j - 1] + (1 if is_pair(seq[i], seq[j]) else 0),  # Pair (i, j)
                max((1 if is_pair(seq[i], seq[t]) else 0) + (1 if is_pair(seq[t + 1], seq[j]) else 0) for t in range(i, j)) # Bifurcation
            )

    # Backtracking to get the dot-bracket notation
    structure = ["." for _ in range(n)]

    def backtrack(i, j, mode):
        if i < j:
            if mode:
                if dp[i][j] == dp[i + 1][j]:  # i is unpaired
                    backtrack(i + 1, j, mode)
                elif dp[i][j] == dp[i][j - 1]:  # j is unpaired
                    backtrack(i, j - 1, mode)
                elif dp[i][j] == dp[i + 1][j - 1] + (1 if is_pair(seq[i], seq[j]) else 0):
                    mode = False
                    if is_pair(seq[i], seq[j]):
                        structure[i] = "("
                        structure[j] = ")"
                    backtrack(i + 1, j - 1, mode)
                else:
                    for t in range(i, j):
                        if dp[i][j] == dp[i][t] + dp[t + 1][j]:
                            backtrack(i, t, mode)
                            backtrack(t + 1, j, mode)
                            break
            else:
                if dp[i][j] == dp[i + 1][j - 1] + (1 if is_pair(seq[i], seq[j]) else 0):
                    if is_pair(seq[i], seq[j]):
                        structure[i] = "("
                        structure[j] = ")"
                    backtrack(i + 1, j - 1, mode)
                elif dp[i][j] == dp[i + 1][j]:  # i is unpaired
                    mode = True
                    backtrack(i + 1, j, mode)
                elif dp[i][j] == dp[i][j - 1]:  # j is unpaired
                    mode = True
                    backtrack(i, j - 1, mode)
                else:
                    for t in range(i, j):
                        if dp[i][j] == dp[i][t] + dp[t + 1][j]:
                            backtrack(i, t, mode)
                            backtrack(t + 1, j, mode)
                            break

    backtrack(0, n - 1, True)
    return "".join(structure)

# Usage
sequence = "GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAAUCUGGAGGUCCUGUGUUCGAUCCACAGAAUUCGCACCA"
dot_bracket = nussinov(sequence)
print(f"Sequence: {sequence}")
print(f"Dot-Bracket Notation: {dot_bracket}")

Sequence: GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAAUCUGGAGGUCCUGUGUUCGAUCCACAGAAUUCGCACCA
Dot-Bracket Notation: (((.(((...((..((...(.((.(((((((((..(().)..).)).))).)..)).)))))))))).)))....
